In [1]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [3]:
# Pull API key from .env file
FMP_API_KEY = config("FMP_API_KEY")
EOD_API_KEY = config("EOD_API_KEY")
# Read in earnings data
earnings_df = pd.read_csv("historical_earnings_new.csv") 
earnings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75540 entries, 0 to 75539
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   earnings_id   75540 non-null  int64  
 1   date          75540 non-null  object 
 2   symbol_id     75540 non-null  int64  
 3   symbol        75540 non-null  object 
 4   eps           71820 non-null  float64
 5   epsEstimated  71419 non-null  float64
 6   time          75539 non-null  object 
dtypes: float64(2), int64(2), object(3)
memory usage: 4.0+ MB


In [4]:
earnings_df

,earnings_id,date,symbol_id,symbol,eps,epsEstimated,time
0,1,11/22/21,0,A,1.21,1.17,amc
1,2,8/17/21,0,A,1.10,0.99,amc
2,3,5/25/21,0,A,0.71,0.83,amc
3,4,2/16/21,0,A,0.94,0.89,amc
4,5,11/23/20,0,A,0.72,0.93,amc
...,...,...,...,...,...,...,...
75535,75536,5/12/16,2377,ZYNE,-0.49,-0.56,bmo
75536,75537,3/14/16,2377,ZYNE,-0.62,-0.29,bmo
75537,75538,11/11/15,2377,ZYNE,-0.66,-0.38,bmo
75538,75539,8/27/15,2377,ZYNE,NaN,NaN,bmo


In [5]:
original_dates = pd.to_datetime(earnings_df["date"])
print(original_dates)

0       2021-11-22
1       2021-08-17
2       2021-05-25
3       2021-02-16
4       2020-11-23
           ...    
75535   2016-05-12
75536   2016-03-14
75537   2015-11-11
75538   2015-08-27
75539   2015-08-05
Name: date, Length: 75540, dtype: datetime64[ns]


In [6]:
from pandas.tseries.offsets import BDay
# Subtract 50 business days to get a series of first dates
first_date = original_dates - BDay(51)
print(first_date)

0       2021-09-10
1       2021-06-07
2       2021-03-15
3       2020-12-07
4       2020-09-11
           ...    
75535   2016-03-02
75536   2016-01-01
75537   2015-09-01
75538   2015-06-17
75539   2015-05-26
Name: date, Length: 75540, dtype: datetime64[ns]


In [7]:
#filtered_df = earnings_df.loc[(pd.to_datetime(earnings_df['date']) > '2012-01-01')]
filtered_df = earnings_df
print(filtered_df)

       earnings_id      date  symbol_id symbol   eps  epsEstimated time
0                1  11/22/21          0      A  1.21          1.17  amc
1                2   8/17/21          0      A  1.10          0.99  amc
2                3   5/25/21          0      A  0.71          0.83  amc
3                4   2/16/21          0      A  0.94          0.89  amc
4                5  11/23/20          0      A  0.72          0.93  amc
...            ...       ...        ...    ...   ...           ...  ...
75535        75536   5/12/16       2377   ZYNE -0.49         -0.56  bmo
75536        75537   3/14/16       2377   ZYNE -0.62         -0.29  bmo
75537        75538  11/11/15       2377   ZYNE -0.66         -0.38  bmo
75538        75539   8/27/15       2377   ZYNE   NaN           NaN  bmo
75539        75540    8/5/15       2377   ZYNE   NaN           NaN  bmo

[75540 rows x 7 columns]


In [8]:
filtered_na_df = filtered_df.dropna()
print(filtered_na_df)

       earnings_id      date  symbol_id symbol   eps  epsEstimated time
0                1  11/22/21          0      A  1.21          1.17  amc
1                2   8/17/21          0      A  1.10          0.99  amc
2                3   5/25/21          0      A  0.71          0.83  amc
3                4   2/16/21          0      A  0.94          0.89  amc
4                5  11/23/20          0      A  0.72          0.93  amc
...            ...       ...        ...    ...   ...           ...  ...
75533        75534  11/14/16       2377   ZYNE -0.67         -0.69  bmo
75534        75535   8/11/16       2377   ZYNE -0.70         -0.54  bmo
75535        75536   5/12/16       2377   ZYNE -0.49         -0.56  bmo
75536        75537   3/14/16       2377   ZYNE -0.62         -0.29  bmo
75537        75538  11/11/15       2377   ZYNE -0.66         -0.38  bmo

[71121 rows x 7 columns]


In [9]:
filtered_na_df.to_csv("filtered_earnings.csv")